In [ ]:

import json
import os
import sys
sys.path.append('../')
from my_packages.data_processing.attributes_processing import transform_test_list
from my_packages.data_processing.split_dataset import read_dataset_to_json
import re

def read(file_path: str):
    with open(file_path, 'r') as file:
        dataset = json.load(file)
    return dataset

def remove_decimals(text: str) -> str:
    """Removes decimal parts from numbers in a string while keeping other text unchanged."""
    return re.sub(r'(-?\d+)\.\d+', r'\1', text)
def extract_func_signature(code_str: str) -> str:
    # Regex explanation:
    # - Start with "func(" (including its parameters) until the opening brace '{'
    # - Then match lazily any text until encountering a line with "in("
    # - Continue matching lazily until encountering a line with "out("
    # - Continue until the first closing '}' that marks the end of the function block.
    pattern = re.compile(
        r'(func\(.*?\{.*?in\(.*?out\(.*?\})',
        re.DOTALL
    )

    match = pattern.search(code_str)
    if match:
        extracted_block = match.group(1)
        print("Extracted block:")
        print(extracted_block)
        return extracted_block
    else:
        print("No match found.")
        return "Not found"
    
def to_inline_string(text: str) -> str:
    """Convert a multi-line string to an inline string with literal \\n characters."""
    # Replace actual newline characters with the literal string "\n"
    return text.replace("\n", r"\n")

main_dataset_folder = '../data/MBPP_Midio_50/MBPP-Midio-50.json'
new_dataset_name = 'UPDATED-MBPP-Midio-50.json'
samples_folder = '../data/MBPP_Midio_50/only_file'
dataset = read_dataset_to_json(main_dataset_folder)

for sample in dataset:
    sample['testing']['python_tests'] = sample['testing']['test_list']
    sample['testing']['tests'] = transform_test_list(sample['testing']['test_list'])
    sample['testing'].pop('test_list')
    code = read(f'{samples_folder}/task_id_{sample["id"]}.midio')
    sample['specification']['function_signature'] = to_inline_string(extract_func_signature(code))
    

print(json.dumps(dataset[0], indent=4))
current_dir = os.getcwd()  # Gets the current working directory
full_path = os.path.join(current_dir, '../data')

os.makedirs(full_path, exist_ok=True)

with open(f'{full_path}/{new_dataset_name}', 'w') as train_file:
    json.dump(dataset, train_file, indent=4)

{
    "prompts": [
        "Create a function that checks whether the given two integers have opposite sign or not.",
        "The flow should create a user-defined function. The body of the function contains two input properties of type Number, a 'Math Expression' node and one output property of type 'Bool'.\n The arrows are connected as follows: The input properties is connected with arrows to 'Math Expression', that contains an expression with the two input properties of type numbers. The 'Math Expression' result is a boolean. This value is connected with arrows to the created output property, that return the Bool from the function. The user-defined function is then created within the main module.",
        "This created user-defined function should then be tested within a seperate module called tests, within the main module. Since the user-defined function does not contain input and ouput triggers, the flow of execution is sent between the assert nodes, using their input and output